In [1]:
!pip install transformers
!pip install tensorboard

In [2]:
import os
from sklearn.metrics import f1_score
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
import pandas as pd
import numpy as np
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from sklearn.model_selection import train_test_split
from datasets import Dataset
import wandb
import warnings
from datasets import Sequence, Value

2025-09-29 17:54:22.320102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759168462.682114      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759168462.787955      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
! unzip ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip;
! unzip ../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip;
! unzip ../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip;
! unzip ../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip;

Archive:  ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
  inflating: train.csv               
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
  inflating: test.csv                
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
  inflating: test_labels.csv         


In [4]:
TRAIN_PATH = '/kaggle/working/train.csv'
VAL_PATH = '/kaggle/working/test.csv'

MODEL_NAME = "unitary/toxic-bert"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
VALID_SIZE = 0.2
TARGET_COL = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
SAMPLE_COL = 'comment_text'
BATCH_SIZE = 32
NUM_LABELS = 6
EPOCHS = 30
PATIENCE = 10

warnings.filterwarnings("ignore")

In [ ]:
API_WANDB = os.getenv("WANDB_API_KEY")
wandb.login(key=API_WANDB)
wandb.init(project="Bert-hf", entity="detect_kaggle")

In [6]:
df = pd.read_csv(TRAIN_PATH)
df = df.sample(frac=0.03, random_state=42)

df_train, df_valid = train_test_split(df, test_size=0.1, shuffle=True)
test_data = pd.read_csv(VAL_PATH)
inference = pd.read_csv('/kaggle/working/sample_submission.csv')

print(
    f'df train shape = {df_train.shape}\n'
    f'df valid shape = {df_valid.shape}\n'
    f'df test shape  = {test_data.shape}\n'
    f'df shape       = {df.shape}\n'
    f'length sum=bmit= {len(inference)}' 
    )

df train shape = (4308, 8)
df valid shape = (479, 8)
df test shape  = (153164, 2)
df shape       = (4787, 8)
length sum=bmit= 153164


In [7]:
# MAX_LEN = int(df.comment_text.str.split().apply(len).max()*1.5)
MAX_LEN = int(df.comment_text.str.split().apply(len).max())
def preprocess_labels(dataframe, target_cols):
    dataframe['labels'] = dataframe[target_cols].values.tolist()
    return dataframe

df_train = preprocess_labels(df_train, TARGET_COL)
df_valid = preprocess_labels(df_valid, TARGET_COL)

df_train.drop(columns=TARGET_COL, inplace=True)
df_valid.drop(columns=TARGET_COL, inplace=True)

df_train

,id,comment_text,labels
10740,1c65ec4affb6ef0a,"""\n\n A Bit Short on Time \n\nHello J.delanoy....","[0, 0, 0, 0, 0, 0]"
123210,93248d52ef3776c9,wrestlecrap is never a good source the book is...,"[0, 0, 0, 0, 0, 0]"
18048,2fac7f1c0cc1839c,Fuck you! You sadistic admin that orgasms when...,"[1, 0, 1, 0, 1, 0]"
50370,86a4ecdc43c131f6,It would also appear from this edit that in or...,"[0, 0, 0, 0, 0, 0]"
81236,d9512c20bef247f3,"""\n\nHow about """"lack of media attention"""" giv...","[0, 0, 0, 0, 0, 0]"
...,...,...,...
54371,91480cba1d94ff98,with four tildes (~~~~),"[0, 0, 0, 0, 0, 0]"
55328,93cccd5620761085,What do you know about schools in Lincolnshire...,"[0, 0, 0, 0, 0, 0]"
151300,7ad5a8b3e2bc5693,"Nirmal\n\nNirmal,\n\n12 FEB, 2012\n\nSubject: ...","[0, 0, 0, 0, 0, 0]"
71906,c087d58f290aa48e,Re: Template:Did you know nominations/2014 Nat...,"[0, 0, 0, 0, 0, 0]"


In [8]:

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)

def tokenize_function(examples):
    return tokenizer(
        examples[SAMPLE_COL],
        padding='max_length',
        truncation=True
    )

train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=SAMPLE_COL,
)
valid_dataset = valid_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=SAMPLE_COL,
)

train_dataset = train_dataset.cast_column("labels", Sequence(Value("float32")))
valid_dataset = valid_dataset.cast_column("labels", Sequence(Value("float32")))


tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/4308 [00:00<?, ? examples/s]

Map:   0%|          | 0/479 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4308 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/479 [00:00<?, ? examples/s]

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    ignore_mismatched_sizes=True,
    problem_type="multi_label_classification",
)


training_args = TrainingArguments(
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    logging_steps=25,
    do_train=True,
    do_eval=True,
    report_to=["tensorboard", "wandb"],
)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))
    preds = (probs > 0.5).astype(int)    
    return {"f1": f1_score(labels, preds, average="macro")}


optimizer = AdamW([
    {'params': list(model.bert.parameters()), 'lr': 1e-5},
    {'params': list(model.classifier.parameters()), 'lr': 1e-3}
])

NUM_STEPS = int(EPOCHS*len(df_train)/BATCH_SIZE)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1*NUM_STEPS),
    num_training_steps=NUM_STEPS
)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    optimizers=(optimizer, scheduler),
    compute_metrics=compute_metrics,
)

In [12]:
trainer.train()

model.save_pretrained("./best_model")
tokenizer.save_pretrained("./best_model")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,F1
1,0.023800,0.024229,0.582059
2,0.022200,0.023598,0.651051
3,0.020600,0.024554,0.709048
4,0.018000,0.025182,0.714800
5,0.015000,0.025207,0.726535


('./best_model/tokenizer_config.json',
 './best_model/special_tokens_map.json',
 './best_model/vocab.txt',
 './best_model/added_tokens.json',
 './best_model/tokenizer.json')

In [38]:

model.eval()

sample_texts = test_data["comment_text"].sample(10).tolist()

enc = tokenizer(
    sample_texts,
    padding=True,
    truncation=True,
    max_length=MAX_LEN,
    return_tensors="pt"
)
enc = {k: v.to(DEVICE) for k, v in enc.items()}

with torch.no_grad():
    outputs = model(**enc)
    probs = torch.sigmoid(outputs.logits)
    preds = (probs > 0.5).int()

for text, p, pred in zip(sample_texts, probs, preds):
    print("Text:", text)
    print("Predicted probabilities:")
    for col, val in zip(TARGET_COL, p.cpu().numpy()):
        print(f"  {col}: {val:.3f}")
    print("Predicted labels:")
    for col, val in zip(TARGET_COL, pred.cpu().numpy()):
        print(f"  {col}: {val}")
    print("---")


Text: ==You== 
 Kiss my ass, bitch.
Predicted probabilities:
  toxic: 0.999
  severe_toxic: 0.244
  obscene: 0.998
  threat: 0.000
  insult: 0.975
  identity_hate: 0.051
Predicted labels:
  toxic: 1
  severe_toxic: 0
  obscene: 1
  threat: 0
  insult: 1
  identity_hate: 0
---
Text: " 

 == Edit Request:Periyar as atheist == 
 First of all, thanks for answering my question. 

 But why should Periyar's views on Islam and Christianity be in Controversies section? On more than one occasion, Periyar has critized organized religions including Islam and Christianity. Is'nt Periyar's statement "" Bane of Tamizhians is Brahmins, Muslims and Christians consider themselves to Tamizhians"" correct place the ""Religion and Atheism section? 

 Below two statements which are the author's own conclusion portray Periyar as favoring Christianity and Islam. 

 Periyar viewed Christianity similar to the monotheistic faith of Islam. He explained that their faith says that there can be only one God which ha